In [2]:
from __future__ import print_function
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

import glob
import os

In [3]:
# Set some standard parameters upfront
pd.options.display.float_format = '{:.1f}'.format
sns.set() # Default seaborn look and feel
plt.style.use('ggplot')
# Same labels will be reused throughout the program
LABELS = ['running_rider', 'scared', 'standing', 'trotting_rider', 'unknown', 'walking_rider', 'head_shake', 'scratch_biting', 'walking_natural', 'grazing', 'running_natural', 'trotting_natural', 'eating', 'jumping', 'shaking', 'rolling', 'fighting', 'rubbing']
# The number of steps within one time segment
TIME_PERIODS = 100
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 50

In [4]:
def read_data(file_path):

    column_names = ['Ax',
                    'Ay',
                    'Az',
                    'Gx',
                    'Gy',
                    'Gz',
                    'Mx',
                    'My',
                    'Mz',
                    'A3D',
                    'G3D',
                    'M3D',
                    'datetime',
                    'label',
                    'segment',
                    'subject']
    df = pd.read_csv(file_path,
                     header=None,
                     names=column_names)
    return df
 
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [5]:
#allfiles = glob.glob('Data/*.csv')
#print(allfiles)
#df = pd.concat((pd.read_data(f) for f in allfiles), ignore_index=True)

df = pd.concat((read_data(f) for f in glob.glob('Data/*')), ignore_index=True)

/home/rosalie/.local/lib/python3.8/site-packages/pandas/core/reshape/concat.py:285: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,14) have mixed types.Specify dtype option on import or set low_memory=False.
  op = _Concatenator(


In [6]:
# Describe the data
show_basic_dataframe_info(df)


Number of columns in the dataframe: 16
Number of rows in the dataframe: 10763585



# PRE PROCESSING

- Shuffle data frame
- Split to test and train set
- Normalize to a range
- TODO segment
- TODO balance

In [39]:
len(df)

10763585

In [7]:
#shuffling the whole dataframe
def shuffle(data_frame):
    return data_frame.sample(frac=1).reset_index(drop=True)


In [8]:
#splitting in to training and test data
def split_test_train(data_frame, ratio):
    ratio = len(data_frame) * ratio
    ratio = int(ratio)
    train = data_frame[:ratio]
    test = data_frame[ratio:]
    return train, test

In [9]:
#im guessing we need some sort of range to adapt the normalization but idk how for now

#normalize the training set
def normalize(train):
    pd.options.mode.chained_assignment = None  # default='warn'
    train['x-axis'] = train['x-axis'] / train['x-axis'].max()
    train['y-axis'] = train['y-axis'] / train['y-axis'].max()
    train['z-axis'] = train['z-axis'] / train['z-axis'].max()
    return train

In [10]:
#segmenting the data
def segment(data_frame, time_step, step, label_name):
    feature_nr = 3
    segments = []
    lables = []
    
    for i in range(0, len(data_frame)):
        xs = data_frame['x-axis'].values[i: i + time_step]
        ys = data_frame['y-axis'].values[i: i + time_step]
        zs = data_frame['z-axis'].values[i: i + time_step]
        segments.append([xs,ys,zs])
        
        label = stats.mode(data_frame[label_name][i: i + time_step])[0][0]
        labels.append(label)
        
    segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_step, N_FEATURES)      
    labels = np.asarray(labels)
    return segments, labels

In [12]:
#balancing the dataframe
def balanced_data_frame(data_frame):
    return -1
    